In [2]:
from urllib.request import Request, urlopen
import json
import pandas as pd
import itertools
from pandas.io.json import json_normalize
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
import dateutil.parser

# Data Preparation

In [2]:
TOKEN = 'Bearer 0/2e807d8f4b098b3f1e1d6bd84a5cf1eb'
POOL = '680563298779433'

In [3]:
def request_json(url):
    request = Request(url)
    request.add_header('Content-Type', 'application/json')
    request.add_header('Authorization', TOKEN)

    data = json.loads(urlopen(request).read().decode('utf-8'))
    return data

In [12]:
data = request_json('https://app.asana.com/api/1.0/projects/{}/tasks?opt_fields=assignee,completed,completed_at,created_at,custom_fields,id,memberships,name,notes,parent,projects,subtasks,tags'.format(POOL))

In [13]:
data_file = open('./data.json', 'w+')

for item in data['data']:
    data_file.write(json.dumps(item) + '\n')

In [33]:
data_file.readlines()

[]

In [5]:
spark = SparkSession.builder.appName("Integration pool spec").getOrCreate()

In [6]:
sc = SparkContext.getOrCreate()

In [7]:
df = spark.read.json('./data.json')
df.columns

['assignee',
 'completed',
 'completed_at',
 'created_at',
 'custom_fields',
 'gid',
 'id',
 'name',
 'notes',
 'parent',
 'projects',
 'subtasks',
 'tags']

In [8]:
get_timestamp = lambda date: dateutil.parser.parse(date).timestamp()
df.filter(df.completed_at > '2019-01-01T00:00:01.112Z').show(1)

+--------------------+---------+--------------------+--------------------+--------------------+---------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|            assignee|completed|        completed_at|          created_at|       custom_fields|            gid|             id|                name|               notes|              parent|            projects|            subtasks|                tags|
+--------------------+---------+--------------------+--------------------+--------------------+---------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|[771731756164271,...|     true|2019-01-02T11:54:...|2018-12-11T23:21:...|[[true,,, 2076489...|943584336303015|943584336303015|[thenorthface][DE...|Atenção: Por gent...|[943562935937591,...|[[680563298779433...|[[943583524227994...|[[1256

In [26]:
df.filter(df.completed == True)

DataFrame[assignee: struct<gid:string,id:bigint,resource_type:string>, completed: boolean, completed_at: string, created_at: string, custom_fields: array<struct<enabled:boolean,enum_options:array<struct<color:string,enabled:boolean,gid:string,id:bigint,name:string,resource_type:string>>,enum_value:struct<color:string,enabled:boolean,gid:string,id:bigint,name:string,resource_type:string>,gid:string,id:bigint,name:string,number_value:double,precision:bigint,resource_subtype:string,resource_type:string,text_value:string,type:string>>, gid: string, id: bigint, name: string, notes: string, parent: struct<gid:string,id:bigint,resource_type:string>, projects: array<struct<gid:string,id:bigint,resource_type:string>>, subtasks: array<struct<gid:string,id:bigint,resource_type:string>>, tags: array<struct<gid:string,id:bigint,resource_type:string>>]